In [ ]:
!pip install transformers sentencepiece datasets

from google.colab import drive
drive.mount('/content/gdrive/')

import os
os.chdir('/content/gdrive/MyDrive/Individual Project')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 9.0 MB/s 
     |████████████████████████████████| 1.2 MB 53.4 MB/s 
     |████████████████████████████████| 365 kB 58.4 MB/s 
     |████████████████████████████████| 6.6 MB 16.0 MB/s 
     |████████████████████████████████| 101 kB 7.7 MB/s 
     |████████████████████████████████| 596 kB 43.4 MB/s 
     |████████████████████████████████| 141 kB 58.6 MB/s 
     |████████████████████████████████| 212 kB 55.3 MB/s 
     |████████████████████████████████| 115 kB 54.2 MB/s 
     |████████████████████████████████| 127 kB 57.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Mounted at /con

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import EarlyStoppingCallback
from transformers import AutoTokenizer, AutoModelWithLMHead
from tokenizers import BertWordPieceTokenizer

In [ ]:
#create a dictionary which associates each string label to an integer value
labels = [ "no", "weak", "strong"]
label2int = dict(zip(labels, list(range(len(labels)))))

# Start building classifier

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "hfl/chinese-xlnet-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
encoded_str = tokenizer("我而家好嬲")
encoded_str

{'input_ids': [7397, 52, 95, 453, 28783, 4, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [ ]:
from transformers import AutoModelForSequenceClassification

num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

Some weights of the model checkpoint at hfl/chinese-xlnet-base were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-xlnet-base and are newly initialized: ['logits_proj.weight', 'logits_proj.bias', 'sequence_summary.summary.bias', 'sequence_summary.summary.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for p

In [ ]:
from transformers import EarlyStoppingCallback

# Early Stopping Module

trainer_callback = EarlyStoppingCallback(early_stopping_patience = 20, early_stopping_threshold = 0.001)

In [ ]:
import torch

from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]


args = TrainingArguments(
    output_dir="/content/gdrive/MyDrive/Individual Project/Model",
    num_train_epochs=50,
    learning_rate =1e-5,
    adam_epsilon=1e-06,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    evaluation_strategy="epoch",
    save_strategy='epoch',
    disable_tqdm=False,
    eval_steps=500,
    logging_steps=500,
    log_level='error',
    save_total_limit = 2,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    overwrite_output_dir=False,
    warmup_steps = 100, # Number of warmup steps
)


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='micro')
    precision = precision_score(y_true=labels, y_pred=pred, average='micro')
    f1 = f1_score(y_true=labels, y_pred=pred, average='micro')

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

data = pd.read_csv("/content/gdrive/MyDrive/Individual Project/EmpathyResponse/Merge_empathy_labelled_ds.csv")

# ----- 1. Preprocess data -----#
# Preprocess data
X = list(data["response"])
y = list(data["empathy_score"])
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

train_ratio = 0.75
validation_ratio = 0.10
test_ratio = 0.15

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1 - train_ratio, random_state=0)

# test is now 15% of the initial data set
# validation is now 10% of the initial data set
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=test_ratio/(test_ratio + validation_ratio), random_state=0) 

X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)
test_dataset=Dataset(X_test_tokenized, y_test)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks = [trainer_callback], # EarlyStoppingCallback module

)

# Train pre-trained model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.769363,0.704545,0.704545,0.704545,0.704545
2,No log,0.689129,0.722727,0.722727,0.722727,0.722727
3,0.601500,0.658446,0.731818,0.731818,0.731818,0.731818
4,0.601500,0.677218,0.736364,0.736364,0.736364,0.736364
5,0.601500,0.874451,0.700000,0.700000,0.700000,0.700000
6,0.486300,0.672002,0.768182,0.768182,0.768182,0.768182
7,0.486300,0.767614,0.740909,0.740909,0.740909,0.740909
8,0.486300,0.712819,0.754545,0.754545,0.754545,0.754545
9,0.370000,0.723583,0.759091,0.759091,0.759091,0.759091
10,0.370000,0.717908,0.759091,0.759091,0.759091,0.759091


TrainOutput(global_step=1760, training_loss=0.4625231829556552, metrics={'train_runtime': 356.9636, 'train_samples_per_second': 49.305, 'train_steps_per_second': 4.93, 'total_flos': 587570448960000.0, 'train_loss': 0.4625231829556552, 'epoch': 10.0})

In [ ]:
tokenizer

PreTrainedTokenizerFast(name_or_path='hfl/chinese-xlnet-base', vocab_size=32000, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '<sep>', 'pad_token': '<pad>', 'cls_token': '<cls>', 'mask_token': '<mask>', 'additional_special_tokens': ['<eop>', '<eod>']})

In [ ]:
trainer.save_model("/content/gdrive/MyDrive/Individual Project/EmpathyResponse/XLNet/MergeXLNetEmpathy1")

In [ ]:
from transformers import XLNetForSequenceClassification

# ----- 3. Predict -----#
# Load test data
# test_data = pd.read_csv("test.csv")
# X_test = list(test_data["review"])
# X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)

# Create torch dataset
# test_dataset = Dataset(X_test_tokenized)
test_dataset = test_dataset

# Load trained model
model_path = "/content/gdrive/MyDrive/Individual Project/EmpathyResponse/XLNet/MergeXLNetEmpathy1"
model = XLNetForSequenceClassification.from_pretrained(model_path, num_labels=3)

# Define test trainer
test_trainer = Trainer(model)

# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)

# raw_pred, _ = model.predictz(test_dataset)
# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)

***** Running Prediction *****
  Num examples = 220
  Batch size = 8


In [ ]:
from sklearn import metrics

print(metrics.classification_report(y_test, y_pred, digits=4))

              precision    recall  f1-score   support

           0     0.8475    0.7042    0.7692        71
           1     0.7125    0.7917    0.7500        72
           2     0.9012    0.9481    0.9241        77

    accuracy                         0.8182       220
   macro avg     0.8204    0.8146    0.8144       220
weighted avg     0.8221    0.8182    0.8171       220

